# Implementation of ML Database Generation

This notebook processes radar data and storm report data and produces a file of radar scan storm objects.

This methodology follows a few important steps:
* Loops through radar scans: each scan is treated as a distinct entity. 
* Calculates storm objects using the TINT package
* Determines if storm is present in the Phoenix metro area (this step may be ignored for other locations)
* Determines if the storm size is small enough to be considered cellular
* Determines if there are reports nearby (spatially and temporally)

### 1. Important packages:

In [ ]:
#For general data handling
import pandas as pd
import numpy as np

#For radar data processing
import pyart

#For date/time handling
from datetime import timedelta
from datetime import datetime as dt
import datetime
from netCDF4 import num2date
import pytz #timezone handling

#For storm tracking:
from tint import Cell_tracks, animate

#For storm reports (including shapefile):
import geopandas as gpd

#For coordinate systems:
from pyproj import Proj

#For mapping (helping with storm reports):
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES

#For handling files
import os

### 2. Important Helper-Functions

These functions are used to standardize the conversion of radar files to grids in PyART.

#### 2.1 Function Parameters

These parameters are passed the the functions below. The comments indicate what aspects of the gridding process they control.

In [ ]:
grid_spacing_v = 35 #normal = 30 - corresponds 586 m (35 corresponds to 500 m)
grid_spacing_h = 201 #normal = 200 - corresponds to roughly 1 km (201 corresponds to 1 km)
grid_interpolation_scheme = "map_gates_to_grid" #map_gates_to_grid best
weighting_function = "Barnes2" #radius of influence scheme
roi_func = "dist" #constant or dist or dist_beam
constant_roi = 500. #for constant radius of influence (not used)
top_level = 17000 #height of top level in meters

#FOR DISTANCE RADIUS OF INFLUENCE
z_factor = 0.05
xy_factor = 0.01
min_radius = 500.

#### 2.2. get_grid function

This function calls the PyART grid_from_radars function with the parameters described above. It then returns a grid object

In [ ]:
def get_grid(radar, vert_grid_spacing = grid_spacing_v, hor_grid_spacing = grid_spacing_h, algo = grid_interpolation_scheme,
            weighting_function = weighting_function, roi_func = roi_func, constant_roi = constant_roi,
            z_factor = z_factor, xy_factor = xy_factor, min_radius = min_radius, top_level = top_level):
    # Returns grid object from radar object.
    grid = pyart.map.grid_from_radars((radar,), grid_shape = (vert_grid_spacing, hor_grid_spacing, hor_grid_spacing),
                                      grid_limits = ((0,top_level),(-100000.0, 100000.0), (-100000.0, 100000.0)),
                                      fields = ['reflectivity'],
                                      gridding_algo = algo, 
                                      weighting_function = weighting_function,
                                      roi_func = roi_func,
                                      constant_roi = constant_roi,
                                      z_factor = z_factor, xy_factor = xy_factor, min_radius = min_radius)
    return grid

#### 2.3. calculate_vil function

This function implements a formula to calculate vertically integrated liquid (VIL) on a grid, and returns the resulting calculated values. 

In [ ]:
def calculate_vil(radar_grid, vert_grid_spacing = grid_spacing_v, top_level = top_level):
    exp = radar_grid/10
    z_new = np.power(10, exp)
    delta_h = top_level/(vert_grid_spacing-1)
    VIL = (3.44*10**(-6))*delta_h*np.sum(np.power(z_new, (4/7)), axis = 0)
    return VIL

#### 2.4. calculate_echo_tops function

This function implements a formula to calculate maximum echo tops on a radar grid for a given threshold, and returns the resulting calculated values

In [ ]:
def calculate_echo_tops(radar_grid, threshold, vert_grid_spacing = grid_spacing_v, top_level = top_level):
    delta_h = top_level/(vert_grid_spacing-1)
    heights = np.arange(0,17001, delta_h)
    heights_arr = np.zeros(np.shape(radar_grid))
    i = 0
    while i < len(heights):
        heights_arr[i, :, :] = heights[i]
        i = i+1
    mask = np.logical_or(radar_grid < threshold, radar_grid.mask)
    heights_masked = np.ma.masked_array(heights_arr, mask = mask)
    ets = (heights_masked.max(axis = 0))
    return ets

### 3. Data preprocessing:

Before radar data and storm reports can be paired into a database of severe and sub-severe storm objects, the different data sources must be prepared. 

#### 3.1. Creating a UTM projection for converting lat/lon data to distances:

This projection will be used later in the calculation of distances between lat/lon points

In [ ]:
myProj = Proj("+proj=utm +zone=12, +ellps=WGS84 +datum=WGS84 +units=m +no_defs") 

#### 3.2. Loading the Phoenix Metro Area shapefile:

This shapefile is a boundary for the Phoenix metro area. This boundary was developed by hand by the NWS Phoenix office. 

In [ ]:
phx_shapefile_path = "./Phoenix Metro.shx"
phx_shapefile = gpd.read_file(phx_shapefile_path)

#### 3.3. Loading and processing the storm reports:

The file (stored in wind_path) contains storm report data within only the region of interest (ex. Maricopa County, AZ). Only certain variables are important to this project (Begin date, time, and location). Additional processing was required:
* The coordinate system was changed to UTM to aid in the use of distances
* Invalid times were removed
* Report times and dates were combined and converted into UTC time

In [ ]:
wind_path = "./storm_data_tswind50.csv"
wind = pd.read_csv(wind_path)[['BEGIN_DATE', 'BEGIN_TIME', 'CZ_TIMEZONE', 'BEGIN_LAT', 'BEGIN_LON', 'MAGNITUDE']]

#changing coordinate system to UTM
wind['UTMx'], wind['UTMy'] = myProj(wind['BEGIN_LON'].values, wind['BEGIN_LAT'].values)
wind = gpd.GeoDataFrame(wind, geometry = gpd.points_from_xy(wind.UTMx,wind.UTMy))

#removing invalid data points
wind_isel =wind['BEGIN_TIME'] > 0
wind = wind[wind_isel]

#processing the date and time
report_time = wind['BEGIN_TIME'][wind_isel] #these are ints
report_date = wind['BEGIN_DATE'][wind_isel] #these are strings

time_true = []
for d, t in zip(report_date, report_time):
    time_str = d + " " + str(t)
    dt_temp = dt.strptime(time_str, "%m/%d/%Y %H%M")
    dt_final = pytz.timezone('US/Arizona').localize(dt_temp)
    dt_final = dt_final.astimezone(pytz.timezone('UTC'))
    time_true.append(dt_final)

wind['true_time'] = time_true

#### 3.4. Locating radar files:

This small loop generates a list of radar scans to utilize. The radar scans are located in the directory given by PATH. The PATH variable should be changed to where radar data is located. 

In [ ]:
radar_scans = []

PATH = "./radar_data_KIWA/2018-9"
for filename in os.listdir(PATH):
    if filename.endswith("_MDM"):continue
    else: radar_scans.append(PATH + "/" + filename)
        
radar_scans.sort()

#### 3.5. Defining final dataframe structure:

In [ ]:
storm_dataset = pd.DataFrame(columns = ['Datetime','lat','lon', 'area', 'vol', 'max_vil',
                                        'max_refl','max_alt','max_et18', 'max_et50', 'max_et60', 'severe_5km',
                                        'severe_10km', 'severe_15km'])

### 4. Database Generation: 

This section generates the database of storm objects paired with storm reports.

#### 4.1. Defining important parameters:

There are several parameters that must be defined. 
* area_threshold: The largest area, in square kilometers, of thunderstorms added to the database. This value is set to 100, removing any large thunderstorm complexes from consideration.
* refl_threshold: The reflectivity requried for a storm object to be considered
* report_threshold_forward: The time (in minutes) after a radar scan to pair a report with the storm. Set to 20 minutes
* report_threshold_backward: The time (in minutes) before a radar scan to pair a report with the storm and classify the storm as 'recently severe'
* radar_buffer: The radius (in kilometers) around the radar to ignore. This is done to remove the influence of the lack of data directly above the radar. 

In [ ]:
area_threshold = 100. #Largest area thunderstorm included (km2)
refl_threshold = 45. #Refl required for storm object
report_threshold_forward = 20. #how many minutes forward to search for reports
report_threshold_backward = -15. #how many minutes backward to search for reports (for third subset)
radar_buffer = 30000. #buffer aroud radar location to ignore storms (in meters)

#### 4.2. Database generation loop

This code block loops through all of the radar scans. For each radar scan (if there are no errors in loading the scan), a radar grid is generated and storm objects are identified. For each storm object:
1. The location is checked to confirm the storm is in the Phoenix metro area
2. The location is checked to confirm the storm is outside of the radar buffer region
3. The size of the storm is checked to ensure it is of small enough size
4. The radar is paired to storm reports within a 5-km, 10-km, and 15-km radius from the storm centroid
5. Storm attributes are calculated and stored in the final dataframe

At the end of the loop, the output is saved to a file. 

In [ ]:
errors = 0
i = 0

for radar_scan in radar_scans:
    
    try:
        grid = get_grid(pyart.io.read_nexrad_archive(radar_scan)) #Try to streamline this
        grids = (g for g in [grid])

        # Calculate storm tracks/ids (using TINT)
        tracks_obj = Cell_tracks()
        tracks_obj.params['FIELD_THRESH'] = refl_threshold
        tracks_obj.get_tracks(grids)

        df = tracks_obj.tracks.copy()
        df = df.reset_index() #df will contain all of our storm information

        if len(df) > 0:

            for uid in df.uid:
                uid_isel = df.uid == uid

                centroid = gpd.points_from_xy([df['lon'][uid_isel]], [df['lat'][uid_isel]])[0]

                centroid_utm_x, centroid_utm_y = myProj([df['lon'][uid_isel].values], [df['lat'][uid_isel].values])
                centroid_utm = gpd.points_from_xy(centroid_utm_x, centroid_utm_y)[0]

                inside_phx = centroid.within(phx_shapefile.geometry[0])

                radar_lat = grid.origin_latitude['data'][0]
                radar_lon = grid.origin_longitude['data'][0]

                radar_utm_x, radar_utm_y = myProj([radar_lon], [radar_lat])

                radar_centroid = gpd.points_from_xy(radar_utm_x,radar_utm_y)[0]
                radar_buf = radar_centroid.buffer(30000.)

                inside_radar_range = centroid_utm.within(radar_buf)

                if inside_phx == True and inside_radar_range == False:

                    if df.area[uid_isel].values[0] < area_threshold:
                        
                        severe_5km = 0
                        severe_10km = 0
                        severe_15km = 0

                        dts = num2date(grid.time['data'], grid.time['units'])
                        dts = pytz.timezone('UTC').localize(dts[0])
                        datestr = dts.strftime('%H:%M UTC %Y-%m-%d')

                        delta_mins = []

                        for d in time_true:
                            delta = d - dts
                            delta_min = delta.days*24*60 + delta.seconds/60
                            delta_mins.append(delta_min)

                        delta_mins = np.array(delta_mins)
                        time_isel = np.logical_and(delta_mins >= 0.0, delta_mins <= report_threshold_forward)
                        time_isel_past = np.logical_and(delta_mins < 0.0, delta_mins >= report_threshold_backward)


                        ###### 5 km radius #######################
                        report_buffer = 5000.

                        if np.sum(time_isel) > 0:
                            report_subset = wind[time_isel]
                            nearby_reports_isel = report_subset.within(centroid_utm.buffer(report_buffer))
                            if np.sum(nearby_reports_isel) > 0:
                                severe_5km = 1

                        if np.sum(time_isel_past) > 0:
                            report_subset_past = wind[time_isel_past]
                            nearby_reports_past_isel = report_subset_past.within(centroid_utm.buffer(report_buffer))
                            if np.sum(nearby_reports_past_isel) > 0:
                                if severe_5km == 0:
                                    severe_5km = -1

                        ###### 10 km radius #######################
                        report_buffer = 10000.

                        if np.sum(time_isel) > 0:
                            report_subset = wind[time_isel]
                            nearby_reports_isel = report_subset.within(centroid_utm.buffer(report_buffer))
                            if np.sum(nearby_reports_isel) > 0:
                                severe_10km = 1

                        if np.sum(time_isel_past) > 0:
                            report_subset_past = wind[time_isel_past]
                            nearby_reports_past_isel = report_subset_past.within(centroid_utm.buffer(report_buffer))
                            if np.sum(nearby_reports_past_isel) > 0:
                                if severe_10km == 0:
                                    severe_10km = -1

                        ###### 15 km radius ########################
                        report_buffer = 15000.

                        if np.sum(time_isel) > 0:
                            report_subset = wind[time_isel]
                            nearby_reports_isel = report_subset.within(centroid_utm.buffer(report_buffer))
                            if np.sum(nearby_reports_isel) > 0:
                                severe_15km = 1

                        if np.sum(time_isel_past) > 0:
                            report_subset_past = wind[time_isel_past]
                            nearby_reports_past_isel = report_subset_past.within(centroid_utm.buffer(report_buffer))
                            if np.sum(nearby_reports_past_isel) > 0:
                                if severe_15km == 0:
                                    severe_15km = -1

                        ##### CALCULATING ATTRIBUTES ##################################################################
                        x_centered = int(df.grid_x[uid_isel].values.round())
                        y_centered = int(df.grid_y[uid_isel].values.round())
                        lon_centered = df.lon[uid_isel].values
                        lat_centered = df.lat[uid_isel].values

                        x = 2

                        radar_subset = grid.fields['reflectivity']['data'][:][:,y_centered-x:y_centered+x, x_centered-x:x_centered+x]

                        # Here are attributes - need to determine what attributes to calculate
                        max_vil = np.max(calculate_vil(radar_subset))
                        max_et18 = np.max(calculate_echo_tops(radar_subset, threshold = 18.5))
                        max_et50 = np.max(calculate_echo_tops(radar_subset, threshold = 50.))
                        max_et60 = np.max(calculate_echo_tops(radar_subset, threshold = 60.))

                        storm_dataset = storm_dataset.append({'Datetime': datestr,'lat':df['lat'][uid_isel].values[0],
                                            'lon':df['lon'][uid_isel].values[0], 'max_vil' : max_vil,'max_et18':max_et18,
                                            'max_refl': df['max'][uid_isel].values[0],'max_alt':df['max_alt'][uid_isel].values[0],
                                            'area':df['area'][uid_isel].values[0], 'vol':df['vol'][uid_isel].values[0],
                                            'max_et50': max_et50, 'max_et60':max_et60, 'severe_5km':severe_5km,
                                            'severe_10km':severe_10km, 'severe_15km':severe_15km}, ignore_index = True)

        print("Done with scan #" + str(i)+": " + radar_scan)

        i = i+1
    
    except:
        print("error in handling radar scan #" + str(i) + ": " + radar_scan)
        errors = errors + 1
        i = i+1
                
# output to file
#storm_dataset.to_csv("storm_dataset_phx201809_full_noradar_varbuffers.csv",sep=',')

### Additional Helpful Information:

#### df columns: 
* scan: always 0
* uid: object id, this is what we iterate over
* time: date and time object (timestamp object):
* grid_x, grid_y: grid locations of centroid
* lon, lat: lat/lon locations of centroid
* area: area of cell
* vol: volume of cell
* max: maximum reflectivity
* max_alt: altitude of maximum reflectivity
* isolated: is the cell isolated

#### Variables in storm dataset:
* date/time: storm time
* lat/lon: storm location
* max_vil: maximum VIL
* max_et18: 18.5 dBZ echo top height
* max_et50: 50 dBZ echo top height
* max_et60: 60 dBZ echo top height
* max_refl: maximum reflectivity (from tint)
* max_alt: height of maximum reflectivity (from tint)
* area: storm area (from tint)
* volume: storm volume (from tint)

#### Lagerquist et al. 2017 variables:
* Used reflectivity at three temperature levels (-20C, -10C, 0C)
* Nine composite radar variables: composite reflectivity, low-level reflectivity, severe hail index (SHI), vertically integrated liquid (VIL), maximum estimated hail size (MESH), midlevel (3-6 km) shear, low-level (0-2 km) shear
* Model sounding data (RUC/NARR):

In [ ]:
#storm_dataset